# Random Forest

## Prior Progress
Random Forest 14 got 70% accuracy on 5 classes (we didn't have class 0 yet).

## Next Task
Do the 6-way classification using Nucleus plus RBC.

In [1]:
import datetime
print(datetime.datetime.now())
from platform import python_version
print('Python',python_version())
import numpy as np
import pandas as pd
import sklearn   # pip install --upgrade scikit-learn
print('sklearn',sklearn.__version__)
import tensorflow as tf
tf.config.list_physical_devices('GPU')

2022-06-08 19:34:02.388153
Python 3.8.10
sklearn 1.0.2


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.feature_selection import RFE
import joblib # used to dump/load sklearn models
from CellProfiler_Util import CP_Util
from RandomForestUtil import RF_Util

In [3]:
BASE_DIR='/home/jrm/ShepherdML/TumorII/'
NUCLEUS_FILES=[
    'Nucleus_Rollup_0.csv',
    'Nucleus_Rollup_1.csv',
    'Nucleus_Rollup_2.csv',
    'Nucleus_Rollup_3.csv',
    'Nucleus_Rollup_4.csv',
    'Nucleus_Rollup_5.csv']
RBC_FILES=[
    'RBC_Rollup_0.csv',
    'RBC_Rollup_1.csv',
    'RBC_Rollup_2.csv',
    'RBC_Rollup_3.csv',
    'RBC_Rollup_4.csv',
    'RBC_Rollup_5.csv']
CLASSES=range(0,6)   # for the first time, use all 6 classes
MODELS_DIR='/home/jrm/Adjeroh/Naved/models/RandomForest.18/'

In [4]:
print(datetime.datetime.now())
PREFIX='abc_'
def set_prefix(prefix):
    global PREFIX
    PREFIX=prefix
def get_prefix(col_name):
    global PREFIX
    return PREFIX+col_name  # hard coded for now
def load_all_classes():
    X = None
    y = None
    for i in CLASSES:
        print(datetime.datetime.now())
        print("Loading class",i)
        Xnuc = pd.read_csv(BASE_DIR+NUCLEUS_FILES[i])
        Xnuc.set_index(['PatchNumber'],inplace=True)
        set_prefix('Nuc_')
        Xnuc.rename(get_prefix,axis='columns',inplace=True)
        Xrbc = pd.read_csv(BASE_DIR+RBC_FILES[i])
        Xrbc.set_index(['PatchNumber'],inplace=True)
        set_prefix('Rbc_')
        Xrbc.rename(get_prefix,axis='columns',inplace=True)
        Xi = Xnuc.join(Xrbc,how='outer')
        #Xi.drop(columns=['PatchNumber'],inplace=True)
        size = len(Xi)
        yi = np.ones(size) * i   # e.g. class 3
        if X is None:
            X = Xi
            y = yi
        else:
            X = pd.concat( (X,Xi) )
            y = np.concatenate( (y,yi) )
    X.fillna(0,inplace=True)  
    return X,y
Xtrain,ytrain=load_all_classes()
Xtrain

2022-06-08 19:34:04.180857
2022-06-08 19:34:04.181162
Loading class 0
2022-06-08 19:34:28.393230
Loading class 1
2022-06-08 19:34:36.876751
Loading class 2
2022-06-08 19:34:46.838128
Loading class 3
2022-06-08 19:34:54.337176
Loading class 4
2022-06-08 19:35:00.642470
Loading class 5


,Nuc_ObjectNumber_count,Nuc_ObjectNumber_mean,Nuc_ObjectNumber_std,Nuc_ObjectNumber_min,Nuc_ObjectNumber_25%,Nuc_ObjectNumber_50%,Nuc_ObjectNumber_75%,Nuc_ObjectNumber_max,Nuc_AreaShape_Area_count,Nuc_AreaShape_Area_mean,...,Rbc_Number_Object_Number_75%,Rbc_Number_Object_Number_max,Rbc_Parent_ShrinkRBC_count,Rbc_Parent_ShrinkRBC_mean,Rbc_Parent_ShrinkRBC_std,Rbc_Parent_ShrinkRBC_min,Rbc_Parent_ShrinkRBC_25%,Rbc_Parent_ShrinkRBC_50%,Rbc_Parent_ShrinkRBC_75%,Rbc_Parent_ShrinkRBC_max
PatchNumber,,,,,,,,,,,,,,,,,,,,,
397,7.0,4.0,2.160247,1.0,2.50,4.0,5.50,7.0,7.0,216.000000,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
398,1.0,1.0,0.000000,1.0,1.00,1.0,1.00,1.0,1.0,200.000000,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
399,1.0,1.0,0.000000,1.0,1.00,1.0,1.00,1.0,1.0,181.000000,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
400,4.0,2.5,1.290994,1.0,1.75,2.5,3.25,4.0,4.0,239.250000,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
401,4.0,2.5,1.290994,1.0,1.75,2.5,3.25,4.0,4.0,237.500000,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1587,7.0,4.0,2.160247,1.0,2.50,4.0,5.50,7.0,7.0,368.285714,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
1588,17.0,9.0,5.049752,1.0,5.00,9.0,13.00,17.0,17.0,519.352941,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
1589,19.0,10.0,5.627314,1.0,5.50,10.0,14.50,19.0,19.0,388.315789,...,1.75,2.0,2.0,1.5,0.707107,1.0,1.25,1.5,1.75,2.0


In [5]:
print(datetime.datetime.now())
print('Do a one-time train/valid split...')
Xt,Xv,yt,yv = train_test_split(Xtrain, ytrain.ravel()) 
        # ,random_state=42) # add this for reproducibility
Xtrain=None
Ytrain=None
print('Xtrain',Xt.shape,'ytrain',yt.shape,'non-zero:',np.count_nonzero(yt))
print('Xvalid',Xv.shape,'yvalid',yv.shape,'non-zero:',np.count_nonzero(yv))
print('Do a one-time fit and evaluate...')
rf1 = RF_Util()
rf1.set_train(Xt,yt)
rf1.set_validation(Xv,yv)
print(datetime.datetime.now())
rf1.fit()
print(datetime.datetime.now())
print('Accuracy:',rf1.validation_accuracy())
print('Confusion:')
print(rf1.validation_confusion())

2022-06-08 19:35:23.552854
Do a one-time train/valid split...
Xtrain (49636, 6392) ytrain (49636,) non-zero: 22652
Xvalid (16546, 6392) yvalid (16546,) non-zero: 7535
Do a one-time fit and evaluate...
2022-06-08 19:35:25.823212
2022-06-08 19:41:17.956718
Accuracy: 81.49401668076877
Confusion:
[[8715  128  135   30    2    1]
 [ 314 1736  417   49    6    0]
 [ 249  436 2041   50    9    9]
 [ 151  280  282  593    2    0]
 [  87  112  152   19  233    0]
 [  50   30   49   13    0  166]]


In [6]:
pd.set_option('display.max_rows', None)
top = rf1.important_features()
top.loc[:100]

,0,1
0,0.007199,Nuc_AreaShape_MaximumRadius_50%
1,0.006719,Nuc_Texture_InfoMeas1_Hematoxylin_5_03_256_mean
2,0.006383,Nuc_Texture_InfoMeas1_Hematoxylin_7_01_256_75%
3,0.006320,Nuc_AreaShape_MeanRadius_mean
4,0.005746,Nuc_AreaShape_MeanRadius_50%
5,0.004994,Nuc_Texture_InfoMeas1_Hematoxylin_7_01_256_50%
6,0.004884,Nuc_Texture_InfoMeas1_Hematoxylin_4_03_256_mean
7,0.004429,Nuc_Texture_InfoMeas1_Hematoxylin_5_01_256_25%
8,0.004406,Nuc_Texture_InfoMeas1_Hematoxylin_4_03_256_75%
9,0.004322,Nuc_Texture_InfoMeas1_Hematoxylin_4_01_256_50%


In [7]:
print(datetime.datetime.now())
joblib.dump(rf1.get_mode(),MODELS_DIR)

2022-06-08 19:41:21.569472


AttributeError: 'RF_Util' object has no attribute 'get_mode'